## Optimization

**Author**: Imad Saddik
<br/>
**Date**: 21/12/2024

---

**Table of contents**<a id='toc0_'></a>    
- [Load the nodes](#toc1_1_)    
- [Pydantic models](#toc1_2_)    
- [Generate graph](#toc1_3_)    
- [Plot the graph in the map](#toc1_4_)    
- [Shortest path](#toc1_5_)    
- [Load the clusters](#toc1_6_)    
- [Remove repeated nodes](#toc1_7_)    
- [Start the optimization](#toc1_8_)    
- [Plot the results](#toc1_9_)    

---

In [ ]:
import psycopg2

connection = psycopg2.connect(
    dbname='routes',
    user='postgres',
    password='postgres',
    host='localhost'
)
connection.autocommit = True
cursor = connection.cursor()

## <a id='toc1_1_'></a>[Load the nodes](#toc0_)

In [3]:
from pydantic import BaseModel


class Node(BaseModel):
    id: int
    latitude: float
    longitude: float
    type_: str

In [4]:
cursor.execute("""
SELECT * FROM nodes
""")

data = cursor.fetchall()
nodes = [Node(id=row[0], latitude=row[1], longitude=row[2], type_=row[3])
         for row in data]

In [5]:
bus_nodes = [node for node in nodes if node.type_ == 'bus']
employee_nodes = [node for node in nodes if node.type_ == 'employee']
company_nodes = [node for node in nodes if node.type_ == 'company']

len(bus_nodes), len(employee_nodes), len(company_nodes)

(40, 500, 1)

In [ ]:
bus_ids = [node.id for node in bus_nodes]
employee_ids = [node.id for node in employee_nodes]
company_ids = [node.id for node in company_nodes]

## <a id='toc1_2_'></a>[Pydantic models](#toc0_)

In [ ]:
class Location(BaseModel):
    latitude: float
    longitude: float


class RouteSegment(BaseModel):
    source_node_id: int
    destination_node_id: int
    distance: float
    duration: float
    coordinates: list[Location]

## <a id='toc1_3_'></a>[Generate graph](#toc0_)

In [ ]:
import networkx as nx


def get_graph(nodes):
    G = nx.DiGraph()

    for i, source_node in enumerate(nodes):
        for j, destination_node in enumerate(nodes):
            if i == j:  # No self-loops
                continue

            source_node_id = source_node.id
            destination_node_id = destination_node.id

            cursor.execute("""
            SELECT * FROM route_mapping 
            WHERE source_node_id = %s AND destination_node_id = %s
            """, (source_node_id, destination_node_id))

            data = cursor.fetchall()
            if not data:
                continue

            row = data[0]
            route_segment = RouteSegment(
                source_node_id=row[0],
                destination_node_id=row[1],
                distance=row[2],
                duration=row[3],
                coordinates=[
                    Location(
                        latitude=latitudeLongitude[0],
                        longitude=latitudeLongitude[1]
                    )
                    for latitudeLongitude in row[4]
                ]
            )
            distance = route_segment.distance
            G.add_edge(i, j, weight=distance)

    return G

## <a id='toc1_4_'></a>[Plot the graph in the map](#toc0_)

In [ ]:
import folium

def generate_graph_map(G, bus_node, assigned_employee_nodes, company_node):
    colors = ['red', 'blue', 'green', 'black']
    nodes = [bus_node] + assigned_employee_nodes + [company_node]
    location_labels = [
        'Bus depot'] + [f'Employee {i}' for i in range(len(assigned_employee_nodes))] + ['Company']

    center_lat = bus_node.latitude
    center_lon = bus_node.longitude
    folium_map = folium.Map(location=[center_lat, center_lon], zoom_start=13)

    for idx, (node, label) in enumerate(zip(nodes, location_labels)):
        color = "red" if idx == 0 else "blue" if idx == len(
            nodes) - 1 else "green"
        folium.Marker(
            [node.latitude, node.longitude],
            popup=label,
            icon=folium.Icon(color=color, icon="info-sign")
        ).add_to(folium_map)

    for i, edge in enumerate(G.edges(data=True)):
        start_idx, end_idx, _ = edge
        start_node = nodes[start_idx]
        end_node = nodes[end_idx]

        start_node_id = start_node.id
        end_node_id = end_node.id

        cursor.execute("""
            SELECT * FROM route_mapping 
            WHERE source_node_id = %s AND destination_node_id = %s
            """, (start_node_id, end_node_id))

        data = cursor.fetchall()
        if not data:
            continue

        row = data[0]
        route = RouteSegment(
            source_node_id=row[0],
            destination_node_id=row[1],
            distance=row[2],
            duration=row[3],
            coordinates=[
                Location(
                    latitude=latitudeLongitude[0],
                    longitude=latitudeLongitude[1]
                )
                for latitudeLongitude in row[4]
            ]
        )

        route_coordinates = [[point.latitude, point.longitude]
                             for point in route.coordinates]

        folium.PolyLine(
            route_coordinates,
            color=colors[i % len(colors)],
            weight=2.5,
            opacity=1,
            tooltip=f"Distance: {route.distance:.2f}m, Duration: {
                route.duration:.2f}s"
        ).add_to(folium_map)

    return folium_map


def generate_optimized_route_map(shortest_path, bus_node, assigned_employee_nodes, company_node):
    colors = ['red', 'blue', 'green', 'black']
    nodes = [bus_node] + assigned_employee_nodes + [company_node]

    # Create the map centered at the bus location
    center_lat = bus_node.latitude
    center_lon = bus_node.longitude
    route_map = folium.Map(location=[center_lat, center_lon], zoom_start=14)

    # Add a marker for the bus depot
    folium.Marker(
        location=[bus_node.latitude, bus_node.longitude],
        icon=folium.Icon(color="green", icon="bus", prefix="fa"),
        popup="Bus Depot"
    ).add_to(route_map)

    # Add markers for employees
    for i, employee_node in enumerate(assigned_employee_nodes, start=1):
        folium.Marker(
            location=[employee_node.latitude, employee_node.longitude],
            icon=folium.Icon(color="blue", icon="user", prefix="fa"),
            popup=f"Employee {employee_node.id}"
        ).add_to(route_map)

    # Add a marker for the company location
    folium.Marker(
        location=[company_node.latitude, company_node.longitude],
        icon=folium.Icon(color="red", icon="briefcase", prefix="fa"),
        popup="Company"
    ).add_to(route_map)

    # Draw the shortest path with numbers
    for i in range(len(shortest_path) - 1):
        start_idx = shortest_path[i]
        end_idx = shortest_path[i + 1]
        start_node = nodes[start_idx]
        end_node = nodes[end_idx]

        start_node_id = start_node.id
        end_node_id = end_node.id

        cursor.execute("""
            SELECT * FROM route_mapping q
            WHERE source_node_id = %s AND destination_node_id = %s
            """, (start_node_id, end_node_id))

        data = cursor.fetchall()
        if not data:
            continue

        row = data[0]
        route = RouteSegment(
            source_node_id=row[0],
            destination_node_id=row[1],
            distance=row[2],
            duration=row[3],
            coordinates=[
                Location(
                    latitude=latitudeLongitude[0],
                    longitude=latitudeLongitude[1]
                )
                for latitudeLongitude in row[4]
            ]
        )

        route_coordinates = [[point.latitude, point.longitude]
                             for point in route.coordinates]

        # Draw the route line
        folium.PolyLine(
            route_coordinates,
            color=colors[i % len(colors)],
            weight=2.5,
            opacity=1,
            tooltip=f"Step: {i}, Distance: {
                route.distance:.2f}m, Duration: {route.duration:.2f}s"
        ).add_to(route_map)

        # Add a number at the midpoint of the route
        midpoint = route_coordinates[len(route_coordinates) // 2]
        folium.Marker(
            location=midpoint,
            icon=folium.DivIcon(
                html=f'''
            <div style="
                display: flex;
                justify-content: center;
                align-items: center;
                font-size: 12px;
                color: black;
                background-color: white;
                padding: 2px 4px;
                border-radius: 3px;
                border: 1px solid black;
                width: 20px;
                height: 20px;
            ">
                <b>{i + 1}</b>
            </div>
            '''
            )
        ).add_to(route_map)

    return route_map

## <a id='toc1_5_'></a>[Shortest path](#toc0_)

In [ ]:
from networkx.algorithms.approximation import traveling_salesman_problem, greedy_tsp


def find_shortest_path(G):
    nodes = list(G.nodes)
    start_node = nodes[0]  # Source (bus location)
    end_node = nodes[-1]   # Sink (company location)

    # Solve TSP for the intermediate nodes
    intermediate_nodes = nodes[:-1]  # Bus node + Employee nodes
    tsp_path = traveling_salesman_problem(
        G.subgraph(intermediate_nodes),
        cycle=False,
        method=greedy_tsp
    )

    # Rearrange the path to ensure it starts at the bus node
    if tsp_path[0] != start_node:
        bus_index = tsp_path.index(start_node)
        tsp_path = tsp_path[bus_index:] + tsp_path[:bus_index]

    # Append the company node as the final destination
    final_path = tsp_path + [end_node]
    return final_path

## <a id='toc1_6_'></a>[Load the clusters](#toc0_)

In [ ]:
import pickle

with open("../data/clusters/bus_assignments.pkl", "rb") as file:
    bus_assignments = pickle.load(file)

bus_assignments

{1: [167,
  448,
  455,
  476,
  283,
  182,
  267,
  207,
  409,
  231,
  376,
  360,
  155,
  261,
  72,
  156,
  45,
  157],
 2: [292,
  431,
  380,
  190,
  341,
  236,
  396,
  340,
  492,
  382,
  466,
  108,
  74,
  505,
  511,
  479,
  55,
  357],
 3: [214,
  297,
  458,
  86,
  384,
  491,
  437,
  496,
  383,
  65,
  159,
  117,
  98,
  163,
  485,
  483,
  333,
  88],
 4: [47,
  517,
  187,
  459,
  345,
  307,
  56,
  152,
  275,
  49,
  125,
  178,
  359,
  82,
  346,
  501,
  138,
  224],
 5: [223,
  286,
  113,
  328,
  457,
  192,
  447,
  205,
  96,
  52,
  116,
  490,
  534,
  43,
  303,
  335,
  58,
  378],
 6: [493,
  281,
  295,
  206,
  472,
  81,
  309,
  334,
  132,
  71,
  516,
  441,
  239,
  451,
  221,
  293,
  84,
  271],
 7: [60,
  73,
  425,
  107,
  480,
  168,
  215,
  265,
  244,
  149,
  410,
  471,
  253,
  506,
  401,
  355,
  395,
  274],
 8: [413,
  290,
  287,
  258,
  229,
  481,
  100,
  530,
  369,
  299,
  164,
  251,
  366,
  538,
  342,
  2

## <a id='toc1_7_'></a>[Remove repeated nodes](#toc0_)

In [ ]:
def make_shortest_path_with_unique_nodes(shortest_path):
    already_assigned = set()

    shortest_path_with_unique_nodes = []
    for node in shortest_path:
        if node not in already_assigned:
            shortest_path_with_unique_nodes.append(node)
            already_assigned.add(node)

    return shortest_path_with_unique_nodes

## <a id='toc1_8_'></a>[Start the optimization](#toc0_)

In [ ]:
from tqdm import tqdm

maps_before = {}
maps_after = {}
optimized_routes = {}

for bus_idx, employee_indices in tqdm(
    bus_assignments.items(),
    total=len(bus_assignments)
):
    if not employee_indices:
        continue

    cursor.execute("""
    SELECT * FROM nodes WHERE id = %s AND type = 'bus'
    """, (bus_idx,))

    data = cursor.fetchall()
    if not data:
        continue

    row = data[0]
    bus_node = Node(id=row[0], latitude=row[1], longitude=row[2], type_=row[3])

    cursor.execute("""
    SELECT * FROM nodes WHERE id = ANY(%s) AND type = 'employee'
    """, (employee_indices,))

    data = cursor.fetchall()
    if not data:
        continue

    assigned_employee_nodes = []
    for row in data:
        employee_node = Node(
            id=row[0], latitude=row[1], longitude=row[2], type_=row[3])
        assigned_employee_nodes.append(employee_node)

    sub_graph_nodes = [bus_node] + assigned_employee_nodes + company_nodes
    G = get_graph(nodes=sub_graph_nodes)
    graph_map = generate_graph_map(
        G=G,
        bus_node=bus_node,
        assigned_employee_nodes=assigned_employee_nodes,
        company_node=company_nodes[0]
    )
    maps_before[bus_idx] = graph_map
    shortest_path = find_shortest_path(G)
    shortest_path = make_shortest_path_with_unique_nodes(shortest_path)
    optimized_routes[bus_idx] = shortest_path
    route_map = generate_optimized_route_map(
        shortest_path=shortest_path,
        bus_node=bus_node,
        assigned_employee_nodes=assigned_employee_nodes,
        company_node=company_nodes[0],
    )
    maps_after[bus_idx] = route_map
    break

  0%|          | 0/40 [00:00<?, ?it/s]


## <a id='toc1_9_'></a>[Plot the results](#toc0_)

In [24]:
maps_before[1]

In [25]:
maps_after[1]

In [27]:
colors = ['red', 'blue', 'green', 'black']
nodes = [bus_node] + assigned_employee_nodes + company_nodes

maps_per_step = {}
shortest_path = optimized_routes[1]
for step in range(len(shortest_path)):
    route_map = folium.Map(
        location=[bus_node.latitude, bus_node.longitude], zoom_start=14)
    folium.Marker(
        location=[bus_node.latitude, bus_node.longitude],
        icon=folium.Icon(color="green", icon="bus", prefix="fa"),
        popup="Bus Depot"
    ).add_to(route_map)

    for i, employee_node in enumerate(assigned_employee_nodes, start=1):
        folium.Marker(
            location=[employee_node.latitude, employee_node.longitude],
            icon=folium.Icon(color="blue", icon="user", prefix="fa"),
            popup=f"Employee {employee_node.id}"
        ).add_to(route_map)

    folium.Marker(
        location=[company_nodes[0].latitude, company_nodes[0].longitude],
        icon=folium.Icon(color="red", icon="briefcase", prefix="fa"),
        popup="Company"
    ).add_to(route_map)

    for i in range(step):
        start_idx = shortest_path[i]
        end_idx = shortest_path[i + 1]
        start_node = nodes[start_idx]
        end_node = nodes[end_idx]

        start_node_id = start_node.id
        end_node_id = end_node.id

        cursor.execute("""
            SELECT * FROM route_mapping 
            WHERE source_node_id = %s AND destination_node_id = %s
            """, (start_node_id, end_node_id))

        data = cursor.fetchall()
        if not data:
            continue

        row = data[0]
        route = RouteSegment(
            source_node_id=row[0],
            destination_node_id=row[1],
            distance=row[2],
            duration=row[3],
            coordinates=[
                Location(
                    latitude=latitudeLongitude[0],
                    longitude=latitudeLongitude[1]
                )
                for latitudeLongitude in row[4]
            ]
        )

        route_coordinates = [[point.latitude, point.longitude]
                             for point in route.coordinates]
        folium.PolyLine(
            route_coordinates,
            color=colors[i % len(colors)],
            weight=2.5,
            opacity=1,
            tooltip=f"Step: {i}, Distance: {
                route.distance:.2f}m, Duration: {route.duration:.2f}s"
        ).add_to(route_map)

        # Add a number at the midpoint of the route
        midpoint = route_coordinates[len(route_coordinates) // 2]
        folium.Marker(
            location=midpoint,
            icon=folium.DivIcon(
                html=f'''
            <div style="
                display: flex;
                justify-content: center;
                align-items: center;
                font-size: 12px;
                color: black;
                background-color: white;
                padding: 2px 4px;
                border-radius: 3px;
                border: 1px solid black;
                width: 20px;
                height: 20px;
            ">
                <b>{i + 1}</b>
            </div>
            '''
            )
        ).add_to(route_map)

    maps_per_step[step] = route_map

In [29]:
for step, route_map in maps_per_step.items():
    display(route_map)